In [1]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/News_Classification

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/News_Classification


In [ ]:
!pip install --pre gql[all]
!pip install tornado==4.5.3

In [3]:
import os
import time
import json

import asyncio
from gql import gql, Client
from gql.transport.aiohttp import AIOHTTPTransport

In [4]:
data_storing_path = os.path.join(os.getcwd(), "dataset")

In [5]:
list_classes = ["Containment & Closure Policies", "Economic Policies", "Health System Policies", "Miscellaneous Policies"]
data_path = os.path.join(data_storing_path, 'result.json')

with open(data_path) as f:
    data_list = json.load(f)

In [6]:
data_list[0]

{'completions': [{'created_at': 1615321919,
   'id': 928001,
   'lead_time': 36.274,
   'result': [{'from_name': 'sentiment',
     'id': 'yQOtNKa4fG',
     'to_name': 'summary',
     'type': 'choices',
     'value': {'choices': ['Miscellaneous Policies']}}]}],
 'data': {'article_id': 1324,
  'content': 'Geoff Cunningham is co-founder and art director of Linksoul, a lifestyle brand that sells a variety of golf products.<n>Cunningham created a golf-inspired graphic, writing "this calls for the low stinger," referring to a low-arching shot.<n>He added a hashtag to the shirts reading #FlattenTheCurve, in an effort to encourage people to flatten the coronavirus curve.<n>The company, which already makes a variety of graphic tees, decided to step up production and donate 100 percent of the shirt\'s sales to the Center for Disaster Philanthropy.',
  'id': 1343027},
 'id': 928}

In [7]:
new_data_list = []
for idx, task in enumerate(data_list):
    try:
      choices_list = task['completions'][0]['result'][0]['value']['choices']
      summary = task['data']['content']
      dataset_id = task['data']['id']
      article_id = task['data']['article_id']
      task_id = task['id']
      
      new_data_list.append({
          "id": dataset_id,
          "article_id": article_id,
          "task_id": task_id,
          "summary": summary,
          "choices": choices_list
      })
    except:
      print(f"Data sample [# {idx}] has data lost.")


Data sample [# 387] has data lost.


In [8]:
async def query_data(url, gql_query):
    transport = AIOHTTPTransport(url=url)

    async with Client(
        transport=transport, fetch_schema_from_transport=True,
    ) as session:

        # Execute single query
        query = gql(gql_query)
        result = await session.execute(query)

    return result

In [9]:
query = """
query getArticles {
    Analysis(order_by: {id: asc}, where: {type: {_eq: "5"}}) {
        id
        content(path: "summary[0]")
        Article {
            content
        }
    }
}
"""

In [13]:
url = ""
query_result = asyncio.run(query_data(url, query))['Analysis']

In [14]:
query_result[0]

{'Article': {'content': '\nA federal detention center at the epicenter of the coronavirus pandemic in New York City has no in-house ability to test sick or high-risk inmates for COVID-19, according to a letter from the jail\'s top official in court documents reviewed by ABC News.\n\n"MCC New York does not have COVID-19 tests," M. Licon-Vitale, the recently installed warden of the Metropolitan Correctional Center in downtown Manhattan, wrote in a letter to a federal judge this week.\n\nThe warden\'s letter came in response to an order from U.S. District Court Judge Paul Engelmayer to test an MCC inmate in his 20s who claimed to be experiencing novel coronavirus symptoms and believed he might have contracted it.\n\n"He was placed into quarantine yesterday after he developed symptoms of COVID-19, including a cough and body aches," wrote Florian Miedel, a defense attorney appointed to represent Bryant Brown, who\'s awaiting trial on murder charges. "We are concerned about MCC\'s ability to

In [16]:
for idx in range(len(new_data_list)):
    dataset_id = new_data_list[idx]['id']

    for idx_ in range(len(query_result)):
        if dataset_id == query_result[idx_]['id']:
            new_data_list[idx]['article'] = query_result[idx_]['Article']['content']

In [18]:
new_data_list[0]

{'article': "Exclusive: How one company’s co-founder helped make thousands of dollars for coronavirus relief. Geoff Cunningham is co-founder and art director of Linksoul, a lifestyle brand that sells a variety of golf products. Fox News spoke to him about his graphic design and how it transformed into a successful COVID-19 fundraiser.Get all the latest news on coronavirus and more delivered daily to your inbox.\xa0Sign up here.\n\xa0\nCountries across the globe are working to “flatten the curve” when it comes to newly reported cases of coronavirus. But one company’s unique interpretation of the popular phrase has unwittingly led to a massive COVID-19 relief fundraiser.Geoff Cunningham is co-founder and art director of Linksoul, a lifestyle brand that sells a variety of golf products. Like most business owners, he revealed to Fox News that he was feeling down and out due to lost business\xa0and having to furlough beloved employees. He even snapped at his social media manager, Lauren, wh

# Uncomment to store `new_data_list`

In [17]:
# new_data_path = os.path.join(data_storing_path, 'processed.json')
# with open(new_data_path, 'w') as f:
#     json.dump(new_data_list, f, indent=4)